In [ ]:
from define import qcode as qc
import numpy as np
from define.QECCLfid import uncorrectable as uc
from define.QECCLfid import minwt as mw
from define import globalvars as gv
from define import chanreps as chrep
import networkx as nx

In [ ]:
def ChangeOrdering(probs, old_order, new_order):
    """
    Change from an ordering in [L,S,T] to another.
    This uses numpy reshape and transpose.
    old = L S T
    new = T L S
    perm = 2 0 1
    """
    t_size = np.power(2, qcode.N - qcode.K, dtype=np.int)
    s_size = t_size
    l_size = np.power(4, qcode.K, dtype=np.int)
    sizes = {"T": t_size, "S": s_size, "L": l_size}
    probs_reshaped = np.reshape(
        probs, [sizes[old_order[0]], sizes[old_order[1]], sizes[old_order[2]]]
    )
    perm = [old_order.index(ax) for ax in new_order]
    probs_reordered = np.reshape(np.transpose(probs_reshaped, perm), -1)
    return probs_reordered

In [ ]:
qcode = qc.QuantumErrorCorrectingCode("11qc")
qc.Load(qcode)
# qc.populate_symplectic(qcode)
#correctable_set = np.array(qcode.Paulis_correctable, dtype=np.int)
# sqprobs = np.random.rand(4)
# sqprobs[0] = 1-0.66**10
# sqprobs[1:] = (1-sqprobs[0]) * sqprobs[1:]/np.sum(sqprobs[1:])
# print(sqprobs[0])
# probs = np.tile(sqprobs, [qcode.N, 1])

In [ ]:
# pauli_probs = np.ones(qcode.PauliOperatorsLST.shape[0], dtype=np.double)
# for i in range(qcode.PauliOperatorsLST.shape[0]):
#     for q in range(qcode.N):
#         pauli_probs[i] *= probs[q, qcode.PauliOperatorsLST[i, q]]
# print(pauli_probs)

In [ ]:
#mw.ComputeUncorrProbs(np.tile(probs, [qcode.N, 1]), qcode, 3)

In [ ]:
#%lprun -f mw.ComputeUncorrProbs mw.ComputeUncorrProbs(np.tile(probs, [qcode.N, 1]), qcode, 3)

In [ ]:
#uncorrs = np.load("/Users/pavi/Documents/chbank/29_05_2020_13_05_35/results/phy_uncorr.npy")
#infids = np.load("/Users/pavi/Documents/chbank/29_05_2020_13_05_35/results/log_infid.npy")

In [ ]:
nstabs = 2**(qcode.N-qcode.K)
nlogs = 4**(qcode.K)
# l, s, t = np.random.randint(0,4,size=(1,)), np.random.randint(0,64,size=(1,)), np.random.randint(0,64,size=(1,))
# tls_order = t * nstabs * nlogs + l * nstabs + s
# lst_order = l * nstabs * nstabs + s * nstabs + t
# print("l = {}, s = {}, t = {}\ntls_order = {}\nlst_order = {}".format(l, s, t, tls_order, lst_order))
# print("pauli_probs[lst_order] = {}\npvec[tls_order] = {}.".format(pauli_probs[lst_order], pvec[tls_order]))

In [ ]:
# freq = np.array(list(map(np.count_nonzero,qcode.Paulis_correctable)),dtype=np.int8)

In [ ]:
# from collections import Counter
# Counter(freq)

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(freq, bins=np.arange(freq.min(), freq.max()+1))

In [ ]:
def GetPolyCoeffs(listOfPaulis,exclude_First_Pauli = 0):
    coeffs = {}
    if exclude_First_Pauli:
        listOfPaulis = listOfPaulis[1:]
    for P in listOfPaulis:
        idcount = np.count_nonzero(P==0)
        non_idcount = len(P) - idcount
        if (idcount,non_idcount) not in coeffs:
            coeffs[(idcount,non_idcount)] = 1
        else:
            coeffs[(idcount,non_idcount)] += 1
    return coeffs    

In [ ]:
def multiplyPols(pol1,pol2):
    res = {}
    for (a,b) in pol1:
        for (c,d) in pol2:
            if (a+c,b+d) not in res:
                res[(a+c,b+d)] = pol1[(a,b)]*pol2[(c,d)]
            else:
                res[(a+c,b+d)] += pol1[(a,b)]*pol2[(c,d)]
    return res    
def multiplyListPols(listOfPols):
    res = {}
    if len(listOfPols)>0:
        res = listOfPols[0]
        for pol in listOfPols[1:]:
            res = multiplyPols(res,pol)
    return res
def addPols(pol1,pol2):
    res = {}
    for (a,b) in pol1:
        if (a,b) in pol2:
            res[(a,b)] = pol1[(a,b)] + pol2[(a,b)]
        else:
            res[(a,b)] = pol1[(a,b)]
    for (a,b) in pol2:
        if (a,b) not in pol1:
            res[(a,b)] = pol2[(a,b)]
    return res

def addListPols(listOfPols):
    res = {}
    if len(listOfPols)>0:
        res = listOfPols[0]
        for pol in listOfPols[1:]:
            res = addPols(res,pol)
    return res

def subPol(pol1,pol2):
    pol2.update((x, y*-1) for x, y in pol2.items())
    res = addPols(pol1,pol2)
    return res

In [ ]:
def GetPolyInd(t,indices):
    nlogs = 4**(qcode.K)
    for L_s in range(4):
            L_t = int(qcode.lookup[t, 0])
            pos_L = qc.PauliProduct(
                np.array([L_t], dtype=np.int), np.array([L_s], dtype=np.int)
            )[0]
            indices[L_s,t*nstabs:(t+1)*nstabs] = pos_L * nstabs * nstabs + np.arange(nstabs) * nstabs + t
            
def GetPolySyndInd(t,indices):
    nlogs = 4**(qcode.K)
    for L_s in range(4):
            indices[L_s,t*nstabs:(t+1)*nstabs] = L_s * nstabs * nstabs + np.arange(nstabs) * nstabs + t


def GetAvgPolyInd(indices):
    for t in range(nstabs):
        GetPolyInd(t,indices)

def GetAvgPoly():
    indices = np.zeros((nlogs,nstabs*nstabs),dtype=np.int)
    GetAvgPolyInd(indices)
    poly_logs = [GetPolyCoeffs(qc.GetOperatorsForLSTIndex(qcode,indices[l])) for l in range(4)]
    return poly_logs

def GetSyndPoly(syndlist):
    poly_logs_synds = [__ for t in range(len(syndlist))]
    for i in range(len(syndlist)):
        t = syndlist[i]
        #print("Syndrome = {}".format(t))
        indices = np.zeros((nlogs,nstabs*nstabs),dtype=np.int)
        GetPolyInd(t,indices)
        # Exclude identity error for t=0
        exclude_First_Pauli = 0
        if(t==0):
            exclude_First_Pauli = 1
        poly_logs_synds[i] = [GetPolyCoeffs(qc.GetOperatorsForLSTIndex(qcode,indices[l,t*nstabs:(t+1)*nstabs]),exclude_First_Pauli) for l in range(4)]
        #     print("Ilog = {}".format(poly_logs_synds[i][0]))
        #     print("Xlog = {}".format(poly_logs_synds[i][1]))
        #     print("Ylog = {}".format(poly_logs_synds[i][2]))
        #     print("Zlog = {}".format(poly_logs_synds[i][3]))
    return poly_logs_synds

def GetSyndProb(syndlist):
    poly_logs_synds = [__ for t in range(len(syndlist))]
    for i in range(len(syndlist)):
        t = syndlist[i]
        #print("Syndrome = {}".format(t))
        indices = np.zeros((nlogs,nstabs*nstabs),dtype=np.int)
        GetPolySyndInd(t,indices)
        # Exclude identity error for t=0
        exclude_First_Pauli = 0
        if(t==0):
            exclude_First_Pauli = 1
        poly_logs_synds[i] = [GetPolyCoeffs(qc.GetOperatorsForLSTIndex(qcode,indices[l,t*nstabs:(t+1)*nstabs]),exclude_First_Pauli) for l in range(4)]
        #     print("Ilog = {}".format(poly_logs_synds[i][0]))
        #     print("Xlog = {}".format(poly_logs_synds[i][1]))
        #     print("Ylog = {}".format(poly_logs_synds[i][2]))
        #     print("Zlog = {}".format(poly_logs_synds[i][3]))
    return poly_logs_synds
    
    


In [ ]:
def Get_unique_elems(lis):
    unique_dict = {}
    for ind,l in enumerate(lis):
        if l not in unique_dict:
            unique_dict[l] = [ind]
        else:
            unique_dict[l].append(ind)
    return sorted(unique_dict.values())

def same_class(i,j,listoflists):
    nlists = len(listoflists)
    for l in listoflists:
        for sublis in l:
            if (i in sublis) and not (j in sublis):
                return 0
    return 1 

def getClasses(listoflists):
    classes = []
    edges = np.zeros((2**(qcode.N-qcode.K),2**(qcode.N-qcode.K)),dtype=np.int)
    visited = np.zeros(2**(qcode.N-qcode.K),dtype=np.int)
    for i in range(2**(qcode.N-qcode.K)):
        for j in range(i+1,2**(qcode.N-qcode.K)):
            if same_class(i,j,listoflists):
                edges[i,j] = 1
                edges[j,i] = 1
    G = nx.from_numpy_matrix(edges)
    comps = nx.connected_components(G)
    for c in sorted(nx.connected_components(G), key=len, reverse=True):
        classes.append(sorted(c))
    return classes
                
                

In [ ]:
poly_logs = GetAvgPoly()
#syndlist = [0,7, 6, 5, 4, 3, 2, 1, 56, 48, 40, 32, 24, 16, 8, 63, 54, 45, 36, 27, 18, 9]

In [ ]:
syndlist = [i for i in range(2**(qcode.N-qcode.K))]
poly_logs_synds = GetSyndPoly(syndlist)

In [ ]:
collect_unique_elems = [__ for __ in range(4**qcode.K)]
for l in range(4**qcode.K):
    #print("l = {}".format(l))
    polynomials = np.zeros((len(syndlist),qcode.N + 1),dtype=np.int)
    for t in range(len(syndlist)):
        for (a,b) in sorted(poly_logs_synds[t][l]):
            polynomials[t,b] = poly_logs_synds[t][l][(a,b)]
        #print(syndlist[t],polynomials[t])
    collect_unique_elems[l] = Get_unique_elems([np.array_str(p) for p in polynomials])
# for l in collect_unique_elems:
#     print("--")
#     print(l)
# print("======")

In [ ]:
classes = getClasses(collect_unique_elems)

In [59]:
# Populate polynomial for average channel
poly_avg = np.zeros((4**qcode.K,qcode.N+1),dtype=np.int)
for l in range(4**qcode.K):
        for (a,b) in poly_logs[l]:
            poly_avg[l,b] =  poly_logs[l][(a,b)]

# Populate polynomial for unnormalised conditional channels
# Populate syndrome probabilities    
# Populate avg * syndrome probs
poly_cls = [np.zeros((4**qcode.K,qcode.N + 1),dtype=np.int) for i in range(len(classes))]
poly_cls_synd = [np.zeros(qcode.N + 1,dtype=np.int) for i in range(len(classes))]
prod_synd_avg = [np.zeros((4**qcode.K,2*(qcode.N) + 1),dtype=np.int) for i in range(len(classes))]
error_synd = [np.zeros((4**qcode.K,2*(qcode.N) + 1),dtype=np.int) for i in range(len(classes))]
for i in range(len(classes)):
    cls = classes[i]
    t = cls[0]
    poly_dict = GetSyndPoly([t])[0]
    for l in range(4):
        for (a,b) in poly_dict[l]:
            poly_cls[i][l,b] =  poly_dict[l][(a,b)]
    synd_prob_dict = addListPols(GetSyndProb([t])[0])
    for (a,b) in synd_prob_dict:
            poly_cls_synd[i][b] +=  synd_prob_dict[(a,b)]
    
    prod_dict = [multiplyPols(synd_prob_dict,poly_logs[l]) for l in range(4)]
    for l in range(4):
        for (a,b) in prod_dict[l]:
            prod_synd_avg[i][l,b] =  prod_dict[l][(a,b)]
    
    sub_avg_prod = [subPol(prod_dict[l],poly_dict[l]) for l in range(4)]
    for l in range(4):
        for (a,b) in sub_avg_prod[l]:
            error_synd[i][l,b] =  sub_avg_prod[l][(a,b)]


In [67]:
print("Total number of classes : ", len(classes))
dominating_order = np.zeros(len(classes),dtype=np.int)
for cl_number in range(len(classes)):
#     print("Class number : ", cl_number)
#     print("Number of errors :", len(classes[cl_number]))
#     print("Syndromes :" , classes[cl_number])
    order_log = np.zeros(4**qcode.K,dtype=np.int)
    for log in range(4**qcode.K):
#         print("Log number: ",log)
#         print("Conditional :")
#         print(poly_cls[cl_number][log])
#         print("--")
#         print("P(s) :")
#         print(poly_cls_synd[cl_number])
#         print("--")
#         print("Average * P(s)")
#         print(prod_synd_avg[cl_number][log])
        diff_arr = np.array(prod_synd_avg[cl_number][log][:len(poly_cls[cl_number][log])])-np.array(poly_cls[cl_number][log])
        order_log[log] = np.nonzero(diff_arr)[0][0]
    dominating_order[cl_number] = np.min(order_log)
#     print("Class number : {} Order : {}".format(cl_number,dominating_order[cl_number]))
print("Order of difference for different syndrome classes :",dominating_order)

Total number of classes :  42
Order of difference for different syndrome classes : [3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 5]


In [68]:
import dill
dill.dump_session('notebook_env.db')

In [2]:
import dill
dill.load_session('notebook_env.db')

In [3]:
for cl_number in range(len(classes)):
    print("Class number : ", cl_number)
    print(np.sum(poly_cls[cl_number],axis=0) - poly_cls_synd[cl_number])

Class number :  0
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  1
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  2
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  3
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  4
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  5
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  6
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  7
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  8
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  9
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  10
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  11
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  12
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  13
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  14
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  15
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  16
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  17
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  18
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  19
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  20
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  21
[0 0 0 0 0 0 0 0 0 0 0 0]
Class number :  22
[

In [ ]:
cls_num = 0
print(sorted(classes[cls_num]))
print("Conditional channel :")
print(poly_cls[cls_num])
print("Syndrome probability :")
print(poly_cls_synd[cls_num])
print("Average channel :")
print(poly_avg)

In [ ]:
suum_cls = poly_cls[0]*len(classes[0])
for cls_num in range(1,7):
    suum_cls += poly_cls[cls_num]*len(classes[cls_num])
print(suum_cls)

In [ ]:
poly_avg

Tracking order of diff / order of channel conditioned on the syndrome

s=0 -- p/1 = p
rest of the classes -- 1/p

In [ ]:
a = (5,2)
print(Ilog[a],Xlog[a],Ylog[a],Zlog[a])

In [ ]:
print("Sgens = {}".format(qcode.S))
print("Tgens = {}".format(qcode.T))

In [ ]:
Ec_trivialdec = qc.GetOperatorsForLSTIndex(qcode,range(nstabs*nstabs))
Ec_minwt = qc.GetOperatorsForLSTIndex(qcode,indices[0])

In [ ]:
poly_l2 = {}
for P in Ec_minwt:
    polyP = multiplyListPols(list(map(lambda i : poly_logs[i],P)))
    poly_l2 = addPols(poly_l2,polyP)

In [ ]:
def calculate_uncorr_l2(p,nqubits,poly_l2,threshold = None):
    l2_contr = 0
    if threshold is not None:
        poly_l2 = dict(((a,b),value) for (a,b), value in poly_l2.items() if b <= threshold)
    for (a,b) in poly_l2:
        l2_contr += np.power(1-p,a)*np.power(p/3,b)*poly_l2[(a,b)]
    return 1-l2_contr

In [ ]:
p = 0.001
uncorr = calculate_uncorr_l2(p,qcode.N,poly_l2)
print(uncorr)

In [ ]:
import matplotlib.pyplot as plt
xaxis = np.linspace(1e-3,5e-2,100)
yaxis = [calculate_uncorr_l2(p,qcode.N,poly_l2) for p in xaxis]
yaxis_trunc = [calculate_uncorr_l2(p,qcode.N,poly_l2,4) for p in xaxis]
plt.plot(xaxis,yaxis,label="exact")
#plt.plot(xaxis,yaxis_trunc,label="truncated")
plt.legend()

In [ ]:
poly_logs[0]

In [ ]:
poly_logs[1][(4,3)]

In [ ]:
poly_logs[1][(4,3)] + poly_logs[2][(4,3)] + poly_logs[3][(4,3)] 

In [ ]:
poly_logs[1][(5,2)] + poly_logs[2][(5,2)] + poly_logs[3][(5,2)] 

In [ ]:
poly_logs[1][(6,1)] + poly_logs[2][(6,1)] + poly_logs[3][(6,1)] 

In [ ]:
classes = [[7, 6, 5, 4, 3, 2, 1],[56, 48, 40, 32, 24, 16, 8],[63, 54, 45, 36, 27, 18, 9],[0]]

In [ ]:
for a in classes:
    print(sorted(a))

In [ ]:
classes[8]